# 1. Combining Poisson Distributions
Given two Poisson distributions:
$$P(x,\mu_A) = \frac{\mu_A^x}{x!}e^{-\mu_A} \text{ and } P(x,\mu_B) = \frac{\mu_B^x}{x!}e^{-\mu_B}$$
Show that they combine to a Poisson distribution:
$$P(x,\mu_C) \text{ where } \mu_C = \mu_A + \mu_B$$

Hint: For any given integer $x$, the one must sum all possibilities of $P(i,\mu_A)P(x-i,\mu_B)$.

# 2. Supernova Light Curve
After a supernova reaches its maximum brightness, the light curve exponentially decays as do the radioactive materials. The decay time can tell us its type. Examine the light curve below.

$$I = [0.921, 0.704, 0.623, 0.550, 0.426, 0.332, 0.258, 0.208, 0.143, 0.130, 0.137, 0.103, 0.058, 0.070, 0.042, 0.060, 0.022, 0.022, 0.011, 0.015]$$
$$ \sigma =  [0.026, 0.048, 0.026, 0.027, 0.068, 0.046, 0.034, 0.017, 0.020, 0.014, 0.015, 0.009, 0.019, 0.010, 0.012, 0.018, 0.007, 0.008, 0.005, 0.005]$$

![](light_curve.PNG)

In [ ]:
I = [0.921, 0.704, 0.623, 0.550, 0.426, 0.332, 0.258, 0.208, 0.143, 0.130, 
     0.137, 0.103, 0.058, 0.070, 0.042, 0.060, 0.022, 0.022, 0.011, 0.015]
sigma =  [0.026, 0.048, 0.026, 0.027, 0.068, 0.046, 0.034, 0.017, 0.020, 0.014, 
          0.015, 0.009, 0.019, 0.010, 0.012, 0.018, 0.007, 0.008, 0.005, 0.005]

## Part(a)
Assuming that $\sigma$ represents a 1-sigma Gaussian uncertainty, find the most likely parameters under the hypothesis that the intensity undergoes an exponential decay:
$$I = I_0 e^{-t/\tau}$$

Here, $\tau$ is the decay time. As one can see, $I_0$ should be nearly unity but, for this problem, do not fix $I_0 = 1$. Calculate the uncertainty in $\tau$. Plot the observations and the fit. 

Hint: One way is to perform a linear fit to $ln(I)$. Be careful how you treat the uncertainty $\sigma$; Taylor expand $ln(I\pm \sigma)$ to calculate the uncertainties of $ln(I)$.